# Resin Identification Code Identifier

Note: This application only supports the identification of plastic objects with resin IDs 1, 2, 5, and 6. It also outputs a guess over the deformation and cleanliness of the object, as well as for whether it has a lid. The lid metric really only works well for simple plastic bottles.

In [1]:
!pip install voila
!jupyter serverextension enable voila --sys-prefix


Enabling: voila
- Writing config: s:\pycharmprojects\plastic_identification.py\venv\etc\jupyter
    - Validating...
      voila 0.2.10 ok


In [2]:
import re
import io
from fastai.vision.all import *
from fastai.vision.widgets import *

pattern = re.compile(r'\d{4}_(a\d\d)b\d\dc\d(d\d)(e\d)(f\d)g\dh\d\.jpg')
def get_y(x):
  y =  list(x for x in pattern.search(str(x)).groups())
  return y

path = Path()
learn_inf = None
try:
    learn_inf = load_learner(path/'models/trained.pkl')
except NotImplementedError:
    import pathlib
    temp = pathlib.PosixPath
    try:
        pathlib.PosixPath = pathlib.PurePosixPath
        learn_inf = load_learner(path/'models/trained.pkl')
    finally:
        pathlib.PosixPath = temp

In [3]:
btn_upload = widgets.FileUpload()
btn_run = widgets.Button()
out_pl = widgets.Output()
lbl_pred = widgets.Label()
labels = {
    'a': widgets.Label(),
    'd': widgets.Label(),
    'e': widgets.Label(),
    'f': widgets.Label()
}

def nice_label(cls, prob):
  t = cls[0]
  d = cls[1:]
  if t == 'a':
    cls = f'Plastic type: ({int(d)}) ' + {
        '00': 'missing or unreadable identifier',
        '01': 'PET - polyethylene terephthalate',
        '02': 'PE-HD - high-density polyethylene',
        '03': 'PVC - polyvinyl chloride',
        '04': 'PE-LD - low-density polyethylene',
        '05': 'PP - polypropylene',
        '06': 'PS - polystyrene',
        '07': 'Other'}[d]
  elif t == 'd':
    cls = 'Deformation: ' + {
        '0': 'undeformed',
        '1': 'small deformation',
        '2': 'medium deformation',
        '3': 'high deformation'}[d]
  elif t == 'e':
    cls = 'Dirtiness: ' + {
        '0': 'clean',
        '1': 'small dirt',
        '2': 'medium dirt',
        '3': 'high dirt'}[d]
  elif t == 'f':
    cls = f'Screw or cap lid: ' + {
        '0': 'lack',
        '1': 'occurs'}[d]
  labels[t].value = cls + f' (Confidence score: {prob:.4f})'
  return cls

def on_click(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl:
        display(img.to_thumb(256, 256))
    pred, pred_idx, probs = learn_inf.predict(img)
    probabilities = list(probs[pred_idx])
    [nice_label(p, probabilities.pop()) for p in pred]
#     lbl_pred.value = f'Prediction: {[nice_label(p) for p in pred]}; #Probability: {prob_fmt}'

btn_upload.observe(on_click, names=['data'])

In [4]:
display(VBox([widgets.Label('Select your image!'), btn_upload, out_pl, *labels.values()]))